In [ ]:
from rich import print

from youtube_summarizer.youtube_scrapper import youtube_scrap

result = youtube_scrap("https://www.youtube.com/watch?v=iYT2haVIgSM")
print(result)

YouTubeScrapperResult(
    id='iYT2haVIgSM',
    thumbnail='https://img.youtube.com/vi/iYT2haVIgSM/maxresdefault.jpg',
    url='https://www.youtube.com/watch?v=iYT2haVIgSM',
    type='video',
    title='Getting Started with the NVIDIA Jetson AGX Thor Developer Kit for Physical AI',
    description='The NVIDIA Jetson AGX Thor Developer Kit is the ultimate developer resource for building the 
future of humanoid robotics and next-generation physical AI applications. Powered by the advanced NVIDIA Blackwell 
GPU and 128 GB of memory,  it delivers up to 2070 FP4 TFLOPS of AI performance—all within a 130 W power envelope. 
\n\nThis video walks you through how to power on and set up the device for the first time (first boot tutorial). It
also covers key features, specifications, and components, and the latest NVIDIA workflows using NVIDIA Isaac GR00T 
N1, the NVIDIA Blueprint for Video Search and Summarization (VSS), and NVIDIA Holoscan Sensor Bridge.\n \n➡️ Learn 
more: NVIDIA Jetson Thor Platform: https://www.nvidia.com/en-us/autonomo... \n\n📝 Read the technical blog post: 
Introducing NVIDIA Jetson Thor, the Ultimate Platform for Physical AI: https://developer.nvidia.com/blog/int... 
\n\n📥 Download NVIDIA Jetpack 7: https://developer.nvidia.com/embedded... \n\n00:05 – Introduction\n01:01 – What’s
in the Box\n01:25 – Hardware Overview\n02:15 – First Power and Boot\n02:54 – The NVIDIA Software Stack\n03:54 – 
Isaac GR00T N1\n4:16 – Video Search and Summarization\n5:04 – NVIDIA Holoscan\n6:05 –Conclusion\n\nNVIDIA, NVIDIA 
Jetson Thor, Physical AI, Humanoid Robots, #NVIDIARobotics, #NVIDIAJetson',
    commentCountInt=None,
    likeCountText='585',
    likeCountInt=585,
    viewCountText='20,850',
    viewCountInt=20850,
    publishDateText='Aug 25, 2025',
    publishDate=datetime.datetime(2025, 8, 25, 8, 1, 11, tzinfo=TzInfo(-07:00)),
    channel=Channel(
        id='UCBHcMCGaiJhv-ESTcWGJPcw',
        url='https://www.youtube.com/@NVIDIADeveloper',
        handle='NVIDIADeveloper',
        title='NVIDIA Developer'
    ),
    chapters=[
        Chapter(title='Introduction', timeDescription='0:05', startSeconds=5),
        Chapter(title='What’s in the Box', timeDescription='1:01', startSeconds=61),
        Chapter(title='Hardware Overview', timeDescription='1:25', startSeconds=85),
        Chapter(title='First Power and Boot', timeDescription='2:15', startSeconds=135),
        Chapter(title='The NVIDIA Software Stack', timeDescription='2:54', startSeconds=174),
        Chapter(title='Isaac GR00T N1', timeDescription='3:54', startSeconds=234),
        Chapter(title='Video Search and Summarization', timeDescription='4:16', startSeconds=256),
        Chapter(title='NVIDIA Holoscan', timeDescription='5:04', startSeconds=304),
        Chapter(title='Conclusion', timeDescription='6:05', startSeconds=365)
    ],
    watchNextVideos=[
        WatchNextVideo(
            id='_2NijXqBESI',
            title="The Physical Turing Test: Jim Fan on Nvidia's Roadmap for Embodied AI",
            thumbnail='https://i.ytimg.com/vi/_2NijXqBESI/hqdefault.jpg?sqp=-oaymwEbCKgBEF5IVfKriqkDDggBFQAAiEIYAXA
BwAEG&rs=AOn4CLDhDFunF4zWqtPXRDocT3z8PGRatg',
            channel=Channel(
                id='UCWrF0oN6unbXrWsTN7RctTw',
                url='https://www.youtube.com/@sequoiacapital',
                handle='sequoiacapital',
                title='Sequoia Capital'
            ),
            publishedTimeText='3 months ago',
            publishedTime=datetime.datetime(2025, 5, 27, 13, 51, 59, 94000, tzinfo=TzInfo(UTC)),
            publishDateText='3 months ago',
            publishDate=datetime.datetime(2025, 5, 27, 13, 51, 59, 94000, tzinfo=TzInfo(UTC)),
            viewCountText='395K views',
            viewCountInt=395000,
            lengthText='17:34',
            lengthInSeconds=1054,
            videoUrl='https://www.youtube.com/watch?v=_2NijXqBESI'
        ),
        WatchNextVideo(
            id='LMPxtcEgtds',
            title='Air Spot | RL Be

In [ ]:
result.title

'Getting Started with the NVIDIA Jetson AGX Thor Developer Kit for Physical AI'

In [ ]:
result.thumbnail

'https://img.youtube.com/vi/iYT2haVIgSM/maxresdefault.jpg'

In [ ]:
result.channel.title

'NVIDIA Developer'

In [ ]:
result.durationFormatted

'00:06:32'

In [ ]:
result.likeCountInt

585

In [ ]:
result.publishDateText

'Aug 25, 2025'

In [ ]:
result.chapters

[Chapter(title='Introduction', timeDescription='0:05', startSeconds=5),
 Chapter(title='What’s in the Box', timeDescription='1:01', startSeconds=61),
 Chapter(title='Hardware Overview', timeDescription='1:25', startSeconds=85),
 Chapter(title='First Power and Boot', timeDescription='2:15', startSeconds=135),
 Chapter(title='The NVIDIA Software Stack', timeDescription='2:54', startSeconds=174),
 Chapter(title='Isaac GR00T N1', timeDescription='3:54', startSeconds=234),
 Chapter(title='Video Search and Summarization', timeDescription='4:16', startSeconds=256),
 Chapter(title='NVIDIA Holoscan', timeDescription='5:04', startSeconds=304),
 Chapter(title='Conclusion', timeDescription='6:05', startSeconds=365)]

In [ ]:
result.transcript

[TranscriptSegment(text='[Music]', startMs='5303', endMs='5840', startTimeText='0:05'),
 TranscriptSegment(text="Hi, I'm Leela with NVIDIA, and this is the Jetson\xa0\nAGX Thor Developer Kit. Jetson Thor is the\xa0\xa0", startMs='5840', endMs='13200', startTimeText='0:05'),
 TranscriptSegment(text="ultimate platform for humanoid robotics. It's\xa0\npart of NVIDIA's three-computer solution for\xa0\xa0", startMs='13200', endMs='19279', startTimeText='0:13'),
 TranscriptSegment(text='accelerating physical AI: NVIDIA DGX for\xa0\ntraining, NVIDIA Omniverse for synthetic\xa0\xa0', startMs='19280', endMs='26400', startTimeText='0:19'),
 TranscriptSegment(text='data generation and physical AI simulation, and\xa0\nNVIDIA Jetson Thor for runtime robotics. Jetson\xa0\xa0', startMs='26400', endMs='34079', startTimeText='0:26'),
 TranscriptSegment(text='AGX Thor is the ideal runtime computer for any\xa0\nkind of physical AI application, from humanoids\xa0\xa0', startMs='34080', endMs='40640', star

In [74]:
import re
from typing import Any, List


def transcript_by_chapter_string(chapters: List[Any], transcript: List[Any]) -> str:
    """Return a single string: '## <Chapter>' followed by its transcript text."""

    # Pre-compile regex patterns for better performance
    whitespace_pattern = re.compile(r"[\xa0\u00a0\u2000-\u200b\u2028\u2029\ufeff\n]")
    multi_space_pattern = re.compile(r" {2,}")

    def _get_value(item, key, default=None):
        """Extract value from object or dict efficiently."""
        if hasattr(item, key):
            return getattr(item, key)
        return item.get(key, default) if isinstance(item, dict) else default

    # Build chapter time windows and pre-allocate segments lists
    windows = []
    for i, ch in enumerate(chapters):
        start_ms = int(_get_value(ch, "startSeconds", 0)) * 1000
        next_start = _get_value(chapters[i + 1], "startSeconds", 10**9) if i + 1 < len(chapters) else 10**9
        end_ms = int(next_start) * 1000

        windows.append({"title": _get_value(ch, "title", f"Chapter {i+1}"), "start_ms": start_ms, "end_ms": end_ms, "segments": []})

    # Single pass assignment of transcript segments to windows
    # Sort transcript by startMs for potential early exit optimization
    sorted_transcript = sorted(transcript, key=lambda x: int(_get_value(x, "startMs", "0")))

    for seg in sorted_transcript:
        seg_ms = int(_get_value(seg, "startMs", "0"))

        # Binary search would be overkill for typical chapter counts, linear search is fine
        for window in windows:
            if window["start_ms"] <= seg_ms < window["end_ms"]:
                window["segments"].append(seg)
                break

    # Build output string with optimized text processing
    blocks = []
    for window in windows:
        if not window["segments"]:
            blocks.append(f"## {window['title']}\n")
            continue

        # Process text in single pass with generator expression
        text_parts = ((_get_value(seg, "text", "") or "").strip() for seg in window["segments"])
        text = " ".join(filter(None, text_parts))

        # Apply regex cleaning
        text = whitespace_pattern.sub(" ", text)
        text = multi_space_pattern.sub(" ", text).strip()

        blocks.append(f"## {window['title']}\n{text}")

    return "\n\n".join(blocks)


chapter_groups = transcript_by_chapter_string(result.chapters, result.transcript)

In [ ]:
import re
from typing import Any, List


def transcript_by_chapter_string(chapters: List[Any], transcript: List[Any]) -> str:
    """Return a single string: '## <Chapter>' followed by its transcript text."""

    # Pre-compile regex patterns for better performance
    whitespace_pattern = re.compile(r"[\xa0\u00a0\u2000-\u200b\u2028\u2029\ufeff\n]")
    multi_space_pattern = re.compile(r" {2,}")

    def _get_value(item, key, default=None):
        """Extract value from object or dict efficiently."""
        if hasattr(item, key):
            return getattr(item, key)
        return item.get(key, default) if isinstance(item, dict) else default

    # Build chapter time windows and pre-allocate segments lists
    windows = []
    for i, ch in enumerate(chapters):
        start_ms = int(_get_value(ch, "startSeconds", 0)) * 1000
        next_start = _get_value(chapters[i + 1], "startSeconds", 10**9) if i + 1 < len(chapters) else 10**9
        end_ms = int(next_start) * 1000

        windows.append({"title": _get_value(ch, "title", f"Chapter {i+1}"), "start_ms": start_ms, "end_ms": end_ms, "segments": []})

    # Single pass assignment of transcript segments to windows
    # Sort transcript by startMs for potential early exit optimization
    sorted_transcript = sorted(transcript, key=lambda x: int(_get_value(x, "startMs", "0")))

    for seg in sorted_transcript:
        seg_ms = int(_get_value(seg, "startMs", "0"))

        # Binary search would be overkill for typical chapter counts, linear search is fine
        for window in windows:
            if window["start_ms"] <= seg_ms < window["end_ms"]:
                window["segments"].append(seg)
                break

    # Build output string with optimized text processing
    blocks = []
    for window in windows:
        if not window["segments"]:
            blocks.append(f"## {window['title']}\n")
            continue

        # Process text in single pass with generator expression
        text_parts = ((_get_value(seg, "text", "") or "").strip() for seg in window["segments"])
        text = " ".join(filter(None, text_parts))

        # Apply regex cleaning
        text = whitespace_pattern.sub(" ", text)
        text = multi_space_pattern.sub(" ", text).strip()

        blocks.append(f"## {window['title']}\n{text}")

    return "\n\n".join(blocks)


chapter_groups = transcript_by_chapter_string(result.chapters, result.transcript)

In [ ]:
chapter_groups

"## Introduction\n[Music] Hi, I'm Leela with NVIDIA, and this is the Jetson AGX Thor Developer Kit. Jetson Thor is the ultimate platform for humanoid robotics. It's part of NVIDIA's three-computer solution for accelerating physical AI: NVIDIA DGX for training, NVIDIA Omniverse for synthetic data generation and physical AI simulation, and NVIDIA Jetson Thor for runtime robotics. Jetson AGX Thor is the ideal runtime computer for any kind of physical AI application, from humanoids to Edge AI agents. It gives you unmatched performance and scalability in a compact, power-efficient form factor. Plus, it's powered by the advanced Blackwell GPU and 128 GB of memory, delivering up to 2070 FP4 TFLOPS of AI compute to effortlessly run the latest generative AI models.\n\n## What’s in the Box\nLet's take a closer look at the Jetson AGX Thor Developer Kit. It includes a Jetson T5000 module, a reference carrier board, an active heat sink with a fan, and a power supply. If you want to set up with a mo

In [ ]:
print(chapter_groups)

## Introduction
[Music] Hi, I'm Leela with NVIDIA, and this is the Jetson AGX Thor Developer Kit. Jetson Thor is the ultimate 
platform for humanoid robotics. It's part of NVIDIA's three-computer solution for accelerating physical AI: NVIDIA 
DGX for training, NVIDIA Omniverse for synthetic data generation and physical AI simulation, and NVIDIA Jetson Thor
for runtime robotics. Jetson AGX Thor is the ideal runtime computer for any kind of physical AI application, from 
humanoids to Edge AI agents. It gives you unmatched performance and scalability in a compact, power-efficient form 
factor. Plus, it's powered by the advanced Blackwell GPU and 128 GB of memory, delivering up to 2070 FP4 TFLOPS of 
AI compute to effortlessly run the latest generative AI models.

## What’s in the Box
Let's take a closer look at the Jetson AGX Thor Developer Kit. It includes a Jetson T5000 module, a reference 
carrier board, an active heat sink with a fan, and a power supply. If you want to set up with a monitor, you'll 
need to provide a computer display, keyboard, and mouse to get started. To make your setup process as smooth as 
possible, we've conveniently

## Hardware Overview
placed all the I/O ports on one side of the board. This thoughtful design choice makes it super easy to connect 
your sensors and manage your wiring when building your robot. Starting here, we see two USB-A ports, an Ethernet 
port, a DisplayPort, an HDMI port, two USB-C ports, a QSFP slot, and a Micro-Fit port for extending the board's 
power if needed. Under the developer kit, we have an M.2 key M slot with a populated 1TB NVMe storage and an M.2 
key E slot with a pre-installed wireless networking card. On the other side of the developer kit, there are three 
buttons for power, force recovery, and reset. It's easy to get started. In the box is a

## First Power and Boot
small booklet that includes a link to step-by-step instructions. First, download the latest NVIDIA JetPack 7 ISO 
image from the JetPack SDK download page and flash it to a USB drive to install on your developer kit. After 
attaching your monitor, keyboard, and mouse, just connect the power supply and your developer kit will power on 
automatically. On first boot, you'll be prompted to choose a username, password, and other basic information. You 
can also set up your Wi-Fi at this time. Jetson Thor delivers a seamless cloud-to-edge

## The NVIDIA Software Stack
experience by running the NVIDIA AI software stack for physical AI applications. This includes NVIDIA Isaac for 
robotics, NVIDIA Metropolis for video analytics AI agents, and NVIDIA Holoscan for sensor processing. With up to 
7.5x the performance of Jetson AGX Orin, Jetson Thor can run all modern AI models from vision-language-action 
models like NVIDIA Isaac GR00T N1 to all popular LLMs and VLMs. NVIDIA Isaac GR00Tis redefining the future of 
robotics with four key pillars: robotic foundational models, synthetic data pipelines, a simulation environment, 
and a runtime computer. You can also accelerate your software development by tethering the developer kit directly 
to any existing robot. Here, NVIDIA Isaac GR00T N1 is running on Thor. To show a developer's end-to-end

## Isaac GR00T N1
journey, we've fine-tuned GR00T N1 for picking up a nut and pouring it in Isaac Sim and Isaac Lab. We've also 
implemented a hardware-in-the-loop scenario running NVIDIA Omniverse on RTX Pro 6000 and inferencing on Jetson 
Thor. The AI blueprint for Video Search and

## Video Search and Summarization
Summarization (VSS) from NVIDIA Metropolis gives you the tools to build and deploy video analytics AI agents that 
can perform contextualized real-time alerts, video summarization, and Q&A by analyzing live camera streams. VSS is 
powering visual agent workforces spanning many use cases, including visual inspection and worker safety in 
manufacturing, fan engagement and player analytics in live sports, and improved emergency response times for 
roadway incidents

In [ ]:
import os

from dotenv import load_dotenv
from google.genai import Client, types
from pydantic import BaseModel, Field
from rich import print

load_dotenv()


class Chapter(BaseModel):
    header: str = Field(description="A descriptive title for the chapter")
    key_points: list[str] = Field(description="Important takeaways and insights from this chapter")
    summary: str = Field(description="A comprehensive summary of the chapter content")


class Analysis(BaseModel):
    title: str = Field(description="The main title or topic of the video content")
    chapters: list[Chapter] = Field(description="Structured breakdown of content into logical chapters")
    key_facts: list[str] = Field(description="Important facts, statistics, or data points mentioned")
    takeaways: list[str] = Field(description="Key insights and actionable takeaways for the audience")
    overall_summary: str = Field(description="A comprehensive summary synthesizing all chapters, facts, and themes")


client = Client(
    api_key=os.getenv("GEMINI_API_KEY"),
    http_options={"timeout": 600000},
)


response = client.models.generate_content_stream(
    model="models/gemini-2.5-pro",
    contents=types.Content(
        parts=[
            types.Part(file_data=types.FileData(file_uri="https://youtu.be/TXjbT6BF6Hc")),
        ]
    ),
    config=types.GenerateContentConfig(
        system_instruction="Analyze the video/transcript according to the schema and follow the original language.",
        temperature=0,
        response_mime_type="application/json",
        response_schema=Analysis,
        thinking_config=types.ThinkingConfig(thinking_budget=2048),
    ),
)


result_parts = [chunk.text for chunk in response if chunk.text is not None and (print(chunk.text, end="") or True)]
final_result = "".join(result_parts)

final_result = Analysis.model_validate_json(final_result)

print(final_result)

In [ ]:
final_result

Analysis(title='Trump Holds Meeting with Zelensky in the Oval Office', chapters=[Chapter(header='Introduction and Welcome', key_points=['Donald Trump welcomes Ukrainian President Volodymyr Zelenskyy to the Oval Office.', 'Trump states that substantial progress is being made in their discussions.', 'He mentions a recent good meeting with the President of Russia and an upcoming meeting with seven powerful European leaders.'], summary="The video begins with a live news report from the White House, where the press is being led into the Oval Office. President Donald Trump is meeting with Ukrainian President Volodymyr Zelenskyy. Trump starts by welcoming Zelenskyy, stating it's an honor to have him. He mentions they've had good discussions and that substantial progress is being made. He also refers to a recent meeting with the President of Russia and a forthcoming meeting with seven European leaders, highlighting the importance of the current discussions."), Chapter(header="Zelenskyy's Remar

In [ ]:
final_result = Analysis.model_validate_json(final_result)

In [ ]:
print(final_result)

Analysis(
    title='美股24小時交易：如何操作、優缺點及應用策略',
    chapters=[
        Chapter(
            header='什麼是美股24小時交易？',
            key_points=[
                '美股可透過電子通訊網絡（ECN）平台實現24小時交易。',
                '除了傳統交易所，還有ARCA、EDGX、Blue Ocean等ECN平台提供延長交易時段。',
                'ECN平台與傳統交易所交易的股票完全相同，只是交易渠道不同。'
            ],
            summary='影片介紹了美股現在可以24小時交易，解決了亞洲投資者需要熬夜的問題。這主要是通過傳統交易所（如紐
約證券交易所、納斯達克）之外的電子通訊網絡（ECN）平台實現的。這些ECN平台，如ARCA、EDGX和Blue 
Ocean，提供了盤前、盤後甚至通宵的交易時段，讓交易時間得以延長。這些平台與傳統交易所提供的股票是完全相同的，只是交易
渠道和時間不同。'
        ),
        Chapter(
            header='24小時美股交易實況操作',
            key_points=[
                '使用IB平台時，需在「Time In Force (TIF)」中選擇「Overnight Trading」。',
                '影片以買入特斯拉股票為例進行操作示範。',
                '下單前務必覆核訂單總金額，防止出錯。',
                '通宵交易時段只接受限價單（Limit Order）。'
            ],
            summary='影片以Interactive Brokers (IB) 
的交易平台為例，逐步示範如何在香港的日間時段（對應美國的夜間）買賣美股。操作關鍵在於下單時，在「Time In Force 
(TIF)」選項中選擇「Overnight 
Trading」。影片展示了如何為特斯拉（TSLA）股票下達一個通宵交易的買單，並強調了檢查訂單總金額（Amount）的重要性，以避
免下錯單。此外，通宵交易目前只接受限價單（Limit Order）。'
        ),
        Chapter(
            header='24小時交易的優點與缺點',
            key_points=[
                '優點：方便，可在亞洲時區對市場新聞作出即時反應。',
                '缺點：流通量低，買賣差價闊。',
                '缺點：訂單稀疏，大額交易容易造成滑價（Trade Slippage）。'
            ],
            summary='24小時交易的最大好處是方便，尤其對於亞洲投資者，可以在白天對突發事件作出反應，無需熬夜。然而，
缺點也很明顯，主要是在非正常交易時段（特別是通宵時段），市場流通量低，導致掛盤稀疏、買賣差價擴大，大額交易容易影響
價格並造成滑價（Slippage）風險。'
        ),
        Chapter(
            header='如何應用24小時交易策略',
            key_points=[
                '真正的價格發現（Price Discovery）發生在正常交易時段。',
                '不應根據通宵時段的價格波動來判斷市場趨勢或做交易決策。',
                '可利用通宵交易時段執行預先設定好的目標價位訂單。'
            ],
            summary='講者分享了他的個人策略。他認為真正的價格發現（Price 
Discovery）發生在交易量最大的正常交易時段（美國白天）。因此，他不會根據通宵時段的價格波動來做新的交易決策。相反，他
會利用這個時段來執行預先設定好的目標價位訂單，例如當股價達到某個預設的賣出價時，就可以掛單賣出。他用街市買魚作比喻
，日間的價格才反映真實的供需，深夜的零星交易價格不具代表性。'
        )
    ],
    key_facts=[
        '美股可透過ECN平台實現24小時交易。',
        'Blue Ocean ATS平台提供美國東岸時間晚上8點至凌晨4點的交易，對應香港夏令時間早上8點至下午4點。',
        '通宵交易時段（Overnight Trading）的流通量極低，僅佔Interactive Brokers總成交量的2%。',
        '盤前盤後（Extended Hours）的成交量約佔總成交量的10%。',
        '通宵交易時段只接受限價單（Limit Order），不接受市價單（Market Order）。'
    ],
    takeaways=[
        '投資者現在可以在香港的辦公時間交易美股，無需熬夜。',
        '進行非正常時段交易時，必須注意流通性低、差價闊和滑價的風險。',
        '利用通宵交易時段執行預設的目標價訂單是較佳的應用方式，而非根據該時段的價格波動進行分析決策。',
        '由於流通量低，應避免在非正常時段進行大額交易。'
    ],
    overall_summary='這部影片詳細介紹了新興的美股24小時交易機制。它解釋了這是通過傳統交易所之外的ECN平台（如Blue 
Ocean）實現的，讓亞洲投資者可以在白天交易美股。影片提供了在Interactive 
Brokers平台上的詳細操作演示，並深入分析了24小時交易的利弊。其主要優點是便利性，但缺點是流通量低和交易成本可能增加（
差價擴大）。講者建議，投資者應理解真正的價格發現主要發生在正常交易時段，因此通宵交易更適合用來執行預先設定好的目標
價訂單，而不是用來判斷市場走勢。'
)

In [ ]:
from youtube_summarizer.youtube_loader import youtube_loader

youtube_loader("https://www.youtube.com/watch?v=S07XJKq-H6Q")

In [ ]:
import os

from dotenv import load_dotenv
from google.genai import Client, types
from pydantic import BaseModel, Field
from rich import print

load_dotenv()


class Chapter(BaseModel):
    header: str = Field(description="A descriptive title for the chapter")
    key_points: list[str] = Field(description="Important takeaways and insights from this chapter")
    summary: str = Field(description="A comprehensive summary of the chapter content")


class Analysis(BaseModel):
    title: str = Field(description="The main title or topic of the video content")
    chapters: list[Chapter] = Field(description="Structured breakdown of content into logical chapters")
    key_facts: list[str] = Field(description="Important facts, statistics, or data points mentioned")
    takeaways: list[str] = Field(description="Key insights and actionable takeaways for the audience")
    overall_summary: str = Field(description="A comprehensive summary synthesizing all chapters, facts, and themes")


client = Client(api_key=os.getenv("GEMINI_API_KEY"))

response = client.models.generate_content(
    model="models/gemini-2.5-pro",
    contents=types.Content(
        parts=[types.Part(text=caption)],
    ),
    config=types.GenerateContentConfig(
        temperature=0,
        response_mime_type="application/json",
        response_schema=Analysis,
        thinking_config=types.ThinkingConfig(thinking_budget=2048),
    ),
)

print(response.text)

{
  "title": "An Investment Analysis of Intel (INTC)",
  "chapters": [
    {
      "header": "Introduction: The Next Big Stock Opportunity",
      "key_points": [
        "The video identifies Intel (INTC) as a stock with a textbook setup for a breakout, despite being down 
60%.",
        "The analysis is supported by institutional buying data from Darkpool trading, which has been consistently 
bullish on Intel.",
        "A free detailed workbook is offered to viewers for a deeper understanding of the company and Wall Street 
trading rules."
      ],
      "summary": "The speaker introduces Intel as a significant investment opportunity, comparing its potential to 
past successes like Palantir. He highlights that institutional investors ('the big boys') have been actively buying
the stock, as indicated by Darkpool data. The video promises to cover five key bullish reasons and a specific entry
point for purchasing the stock."
    },
    {
      "header": "The Five Bullish Reasons for Intel",
      "key_points": [
        "The US government is providing Intel with approximately $7.8 billion in grants to build domestic chip 
foundries, making it a critical piece of US infrastructure.",
        "New leadership is targeting $10 billion in cost savings and launching more competitive chips that are 
allegedly 50% cheaper to run than Nvidia's.",
        "The rise of AI is expected to drive a PC refresh cycle, as users will need more powerful machines for 
local computing, benefiting Intel.",
        "Intel is investing tens of billions to build a foundry business to rival TSMC and capture a share of the 
$80 billion third-party foundry market.",
        "Based on its current valuation, some analysts project a potential 3-5x return in the next few years if the
company successfully executes its turnaround plan."
      ],
      "summary": "This chapter details the five core fundamental arguments for being bullish on Intel. These 
include significant government support to onshore chip manufacturing, aggressive cost-cutting measures under new 
leadership, the growing demand for AI-capable PCs, the strategic expansion into the third-party foundry business to
compete globally, and the stock's attractive valuation which suggests a high potential for future returns."
    },
    {
      "header": "Technical Analysis and Entry Strategy",
      "key_points": [
        "The stock has been in a sideways consolidation phase for about a year, suggesting that most sellers have 
already exited their positions.",
        "Recent trading volume has been historically high, indicating a 'clean slate' is being created for a 
potential upward move.",
        "A key resistance level is identified at $25, where market makers and institutions are likely to sell.",
        "The recommended entry point is not now, but after the stock breaks out above the recent high of 
approximately $27.50 on significant volume."
      ],
      "summary": "The speaker shifts from fundamentals to technical analysis, examining Intel's stock chart. He 
notes a year-long consolidation period, which he views as a positive sign as it shakes out weak hands. He points 
out a significant resistance level at $25. The core message is to avoid buying prematurely and instead wait for a 
confirmed breakout above the $27.50 level, supported by high trading volume, as this would signal strong buying 
momentum and a higher probability of a sustained upward trend."
    },
    {
      "header": "Conclusion and Risk Management",
      "key_points": [
        "Investors should not blindly buy a stock based on a YouTube video or FOMO (Fear Of Missing Out).",
        "It is crucial to have a personal risk management strategy, including setting stop-losses.",
        "The provided workbook can help investors develop their own strategy and understand the rules for buying 
such stocks."
      ],
      "summary": "The video concludes with a strong caution against impulsive buying. The speaker emphasizes t